In [7]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)



In [8]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='"Your time is limited, so don’t waste it living someone else’s life.\n\nDon’t be trapped by dogma — which is living with the results of other people’s thinking. Don’t let the noise of others’ opinions drown out your own inner voice. And most important, have the courage to follow your heart and intuition.\n\nThey somehow already know what you truly want to become. Everything else is secondary.\n\nWhen I was 17, I read a quote that went something like:\nIf you live each day as if it was your last, someday you’ll most certainly be right."')]

In [11]:
embeddings=OllamaEmbeddings()
db=FAISS.from_documents(docs,embeddings)
db

In [12]:
#### querying

query="At what age did the speaker read the quote about living each day as if it were your last?"
docs=db.similarity_search(query)
docs

[Document(id='e8ca016e-b8f0-457a-94e2-53f634eb18fd', metadata={'source': 'speech.txt'}, page_content='"Your time is limited, so don’t waste it living someone else’s life.\n\nDon’t be trapped by dogma — which is living with the results of other people’s thinking. Don’t let the noise of others’ opinions drown out your own inner voice. And most important, have the courage to follow your heart and intuition.\n\nThey somehow already know what you truly want to become. Everything else is secondary.\n\nWhen I was 17, I read a quote that went something like:\nIf you live each day as if it was your last, someday you’ll most certainly be right."')]

In [13]:
docs[0].page_content

'"Your time is limited, so don’t waste it living someone else’s life.\n\nDon’t be trapped by dogma — which is living with the results of other people’s thinking. Don’t let the noise of others’ opinions drown out your own inner voice. And most important, have the courage to follow your heart and intuition.\n\nThey somehow already know what you truly want to become. Everything else is secondary.\n\nWhen I was 17, I read a quote that went something like:\nIf you live each day as if it was your last, someday you’ll most certainly be right."'

In [15]:
### retriever
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'"Your time is limited, so don’t waste it living someone else’s life.\n\nDon’t be trapped by dogma — which is living with the results of other people’s thinking. Don’t let the noise of others’ opinions drown out your own inner voice. And most important, have the courage to follow your heart and intuition.\n\nThey somehow already know what you truly want to become. Everything else is secondary.\n\nWhen I was 17, I read a quote that went something like:\nIf you live each day as if it was your last, someday you’ll most certainly be right."'

### Similarity Search with Score 

There are some FAISS specific methods.One of them is similarity_search_with_score, which allows you return not only the documents but also the distance score of the query to them. The returned distance score is 
L2 distance.Therefore, a lower score is better

In [16]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='e8ca016e-b8f0-457a-94e2-53f634eb18fd', metadata={'source': 'speech.txt'}, page_content='"Your time is limited, so don’t waste it living someone else’s life.\n\nDon’t be trapped by dogma — which is living with the results of other people’s thinking. Don’t let the noise of others’ opinions drown out your own inner voice. And most important, have the courage to follow your heart and intuition.\n\nThey somehow already know what you truly want to become. Everything else is secondary.\n\nWhen I was 17, I read a quote that went something like:\nIf you live each day as if it was your last, someday you’ll most certainly be right."'),
  np.float32(17565.064))]

In [17]:

embedding_vector=embeddings.embed_query(query)
embedding_vector



[0.8658106327056885,
 1.3625593185424805,
 -0.18905961513519287,
 1.0666197538375854,
 -0.8751623034477234,
 1.8226069211959839,
 2.008117914199829,
 0.6283872723579407,
 0.013532931916415691,
 -2.234071731567383,
 1.5556058883666992,
 -0.4395708739757538,
 -1.6000314950942993,
 2.4726755619049072,
 -0.2588511109352112,
 -0.6729392409324646,
 0.03479238972067833,
 0.36433306336402893,
 -0.3150979280471802,
 -2.194483518600464,
 -0.9150460362434387,
 -0.9686644673347473,
 2.0449929237365723,
 -2.6976139545440674,
 0.09230893850326538,
 -2.579744338989258,
 2.6974050998687744,
 0.031200584024190903,
 0.8171014785766602,
 -1.2901363372802734,
 1.4182374477386475,
 -1.1206625699996948,
 -0.18293406069278717,
 3.1217238903045654,
 1.0376065969467163,
 -3.009772777557373,
 -0.5917271375656128,
 0.10638047009706497,
 -1.8430492877960205,
 -0.5644927024841309,
 0.12225770205259323,
 -3.563220977783203,
 0.6952771544456482,
 -0.44406944513320923,
 -1.773694634437561,
 0.7643252611160278,
 -0.23

In [18]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='e8ca016e-b8f0-457a-94e2-53f634eb18fd', metadata={'source': 'speech.txt'}, page_content='"Your time is limited, so don’t waste it living someone else’s life.\n\nDon’t be trapped by dogma — which is living with the results of other people’s thinking. Don’t let the noise of others’ opinions drown out your own inner voice. And most important, have the courage to follow your heart and intuition.\n\nThey somehow already know what you truly want to become. Everything else is secondary.\n\nWhen I was 17, I read a quote that went something like:\nIf you live each day as if it was your last, someday you’ll most certainly be right."')]

In [22]:
##Saving and Loading

db.save_local("faiss_index")

In [27]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [28]:
docs

[Document(id='e8ca016e-b8f0-457a-94e2-53f634eb18fd', metadata={'source': 'speech.txt'}, page_content='"Your time is limited, so don’t waste it living someone else’s life.\n\nDon’t be trapped by dogma — which is living with the results of other people’s thinking. Don’t let the noise of others’ opinions drown out your own inner voice. And most important, have the courage to follow your heart and intuition.\n\nThey somehow already know what you truly want to become. Everything else is secondary.\n\nWhen I was 17, I read a quote that went something like:\nIf you live each day as if it was your last, someday you’ll most certainly be right."')]